# Проект "SQL"

## Синопсис проекта

В нашем распоряжении база данных с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг, которую в дальнейшем планируется использовать для создания специализированного сервиса.

## Задача проекта

- Необходимо подключиться к базе данных
- Сделать ряд SQL запросов, чтобы проверить содержание, корректность структуры и работоспособности базы данных.

## Описание базы данных

**Таблица** `books`

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица** `authors`

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица** `publishers`

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица** `ratings`

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица** `reviews`

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Формулировки заданий по запросам к базе данных

- **Задание 1:** Посчитать, сколько книг вышло после 1 января 2000 года;
- **Задание 2:** Для каждой книги посчитать количество обзоров и среднюю оценку;
- **Задание 3:** Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
- **Задание 4:** Определить автора с самой высокой средней оценкой книг — должны учитываться только книги с 50 и более оценками;
- **Задание 5:** Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Блок библиотек и функций для работы с проектом

In [8]:
import pandas as pd
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:,.2f}'.format)


# функция для получения результата запроса
def view(result):
        return pd.io.sql.read_sql(result, con=engine)   
    
# функция для вывода информации о результате запроса, если результат запроса dataframe
def show_df(df):
    return (
            display(df.head()),
            display(df.describe()),
            df.info()
               )

# функция для вывода информации о результате запроса, если результат запроса какое либо отдельное значение
def show_query(df):
        return (
               display(df.head())
               )

## Подключение к базе данных

In [9]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Исследование таблиц базы данных

На первом этапе получим информацию об имеющихся в нашем распоряжении таблицах базы данных

### Таблица `books`

In [10]:
query = ''' SELECT *
            FROM books
        '''
data_table = view(query)
info = show_df(data_table)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,book_id,author_id,num_pages,publisher_id
count,"1,000.00","1,000.00","1,000.00","1,000.00"
mean,500.50,320.42,389.11,171.27
std,288.82,181.62,229.39,99.08
min,1.00,1.00,14.00,1.00
25%,250.75,162.75,249.00,83.00
50%,500.50,316.50,352.00,177.50
75%,750.25,481.00,453.00,258.00
max,"1,000.00",636.00,"2,690.00",340.00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


**Вывод** 

- В таблице сведения о 1000 книгах
- Типы данных корректны, за исключением столбца `publication_date`, при работе с которым будет необходимо преобразовывать тип данных из `object` в `datetime`
- пропусков в данных нет
- каких то иных нарушений не выявлено

### Таблица `authors`

In [11]:
query = ''' SELECT *
            FROM authors
        '''
data_table = view(query)
info = show_df(data_table)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,author_id
count,636.00
mean,318.50
std,183.74
min,1.00
25%,159.75
50%,318.50
75%,477.25
max,636.00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


**Вывод**

- В таблице сведения о 636 авторах
- Типы данных корректны
- Пропусков в данных нет
- Каких то иных нарушений не выявлено

### Таблица `publishers`

In [12]:
query = ''' SELECT *
            FROM publishers
        '''
data_table = view(query)
info = show_df(data_table)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,publisher_id
count,340.00
mean,170.50
std,98.29
min,1.00
25%,85.75
50%,170.50
75%,255.25
max,340.00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


**Вывод**

- В таблице сведения о 340 издательствах
- Типы данных корректны
- Пропусков в данных нет
- Каких то иных нарушений не выявлено

### Таблица `ratings`

In [13]:
query = ''' SELECT *
            FROM ratings
        '''
data_table = view(query)
info = show_df(data_table)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,rating_id,book_id,rating
count,"6,456.00","6,456.00","6,456.00"
mean,"3,228.50",510.57,3.93
std,"1,863.83",284.14,0.94
min,1.00,1.00,1.00
25%,"1,614.75",291.00,3.00
50%,"3,228.50",506.00,4.00
75%,"4,842.25",750.00,5.00
max,"6,456.00","1,000.00",5.00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


**Вывод**

- В таблице сведения о 6456 событиях, когда пользователи выставляли определенной книге личный рейтинг
- Типы данных корректны
- Пропусков в данных нет
- Каких то иных нарушений не выявлено

### Таблица `reviews`

In [14]:
query = ''' SELECT *
            FROM reviews
        '''
data_table = view(query)
info = show_df(data_table)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


,review_id,book_id
count,"2,793.00","2,793.00"
mean,"1,397.00",504.69
std,806.41,288.47
min,1.00,1.00
25%,699.00,259.00
50%,"1,397.00",505.00
75%,"2,095.00",753.00
max,"2,793.00","1,000.00"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


**Вывод**

- В таблице сведения о 2793 обзорах на книги, содержащиеся в базе данных
- Типы данных корректны
- Пропусков в данных нет
- Каких то иных нарушений не выявлено

## Исследование работоспособности базы данных

### Запрос 1

**Формулировка задания** Посчитать, сколько книг вышло после 1 января 2000 года

**Комментарий к формулировке запроса** 
- Формируя запрос исходил из логики, что мы должны получить данные обо всех книгах вышедших после 1 января 2000 года, включая 1 января 2000 года 
- Выведены сведения о полученном количестве книг

In [15]:
query = '''        
        SELECT 
               COUNT(book_id) as count_books
        FROM books
        WHERE CAST(publication_date as date) >= '2000-01-01'
        '''

data_table = view(query)
info = show_query(data_table)

,count_books
0,821


**Вывод** В базе данных содержится 821 книга вышедшие 1 января 2000 года и позднее этой даты

### Запрос 2

**Формулировка задания** Для каждой книги посчитать количество обзоров и среднюю оценку

**Комментарий к формулировке запроса** Запрос сформирован в следующем порядке:
- сформирована временная таблица содержащая сведения о среднем рейтинге всех книг
- сформирована временная таблица содержащая сведения о количестве ревью на каждую книгу
- сведения из двух временных таблиц присоединены к сведениям об идентификаторе и наименовании каждой книги
- выведены сведения о 5 первых строках получившейся таблицы

In [25]:
query = '''        
        WITH 
        i AS (SELECT book_id, 
                     AVG(rating) as avg_rating
              FROM ratings
              GROUP BY book_id),
              
        j AS (SELECT book_id, 
                     COUNT(review_id) as count_review
              FROM reviews
              GROUP BY book_id)
        
        SELECT 
               b.book_id as book_id,
               b.title as book_title,
               ROUND(rat.avg_rating, 2) as avg_rating,
               rev.count_review
        FROM books as b
        LEFT JOIN i as rat ON b.book_id = rat.book_id
        LEFT JOIN j as rev ON b.book_id = rev.book_id
        
        '''

data_table = view(query)
info = show_df(data_table)

,book_id,book_title,avg_rating,count_review
0,652,The Body in the Library (Miss Marple #3),4.50,2.00
1,273,Galápagos,4.50,2.00
2,51,A Tree Grows in Brooklyn,4.25,5.00
3,951,Undaunted Courage: The Pioneering First Missio...,4.00,2.00
4,839,The Prophet,4.29,4.00


,book_id,avg_rating,count_review
count,"1,000.00","1,000.00",994.00
mean,500.50,3.90,2.81
std,288.82,0.56,1.06
min,1.00,1.50,1.00
25%,250.75,3.50,2.00
50%,500.50,4.00,3.00
75%,750.25,4.33,3.00
max,"1,000.00",5.00,7.00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   book_id       1000 non-null   int64  
 1   book_title    1000 non-null   object 
 2   avg_rating    1000 non-null   float64
 3   count_review  994 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 31.4+ KB


**Вывод**
- На 6 книг отсутствуют обзоры
- Максимальное количество обзоров на 1 книгу - 7, в среднем на 1 книгу читатели оставляли немногим менее 3 обзоров
- средний рейтинг книги - 3,9

### Запрос 3

**Формулировка задания** Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

**Комментарий к формулировке запроса** Запрос сформирован в следующем порядке:
- сформирована временная таблица содержащая сведения о количестве изданных книг по каждому идентификатору издателя
- временная таблица присоединена к таблице с наименованиями издателей
- выведены сведения об издателе с максимальным количеством изданных книг

In [17]:
query = '''        
        WITH 
       
        i AS (SELECT publisher_id, 
                     COUNT(book_id) as count_books
              FROM books
              WHERE num_pages > 50
              GROUP BY publisher_id)
        
        SELECT 
               p.publisher,
               i.count_books
               
        FROM publishers as p
        JOIN i ON i.publisher_id = p.publisher_id
        ORDER BY i.count_books DESC
        LIMIT 1
        
        '''

data_table = view(query)
info = show_query(data_table)

,publisher,count_books
0,Penguin Books,42


**Вывод** Наибольше количество книг (42) выпустило издательство `Penguin Books`

### Запрос 4

**Формулировка задания** Определить автора с самой высокой средней оценкой книг — должны учитываться только книги с 50 и более оценками

**Комментарий к формулировке запроса** Запрос сформирован в следующем порядке:
- сформирована временная таблица содержащая сведения о среднем рейтинге каждой книги, оставлены только книги, которым выставлено более 50 оценок
- сформирована временная таблица содержащая сведения о среднем рейтинге всех книг одного автора
- временная таблица присоединена к таблице с именами авторов
- выведены сведения об авторе с наибольшим средним показателем

In [29]:
query = '''        
        WITH 
        b_rating AS (SELECT book_id,
                            COUNT(rating_id) as count_score,
                            AVG(rating) as avg_rating_books
                     FROM ratings           
                     GROUP BY book_id
                     HAVING COUNT(rating_id) > 50),
       
        a_rating AS (SELECT b.author_id, 
                            AVG(br.avg_rating_books) as avg_rating
                     FROM books as b
                     JOIN b_rating as br ON br.book_id=b.book_id
                     GROUP BY b.author_id)
        
        SELECT 
               a.author,
               ar.avg_rating as avg_rating_books
              
        FROM authors as a
        JOIN a_rating as ar ON ar.author_id = a.author_id
        ORDER BY ar.avg_rating DESC
        LIMIT 1
        
        '''

data_table = view(query)
info = show_query(data_table)

,author,avg_rating_books
0,J.K. Rowling/Mary GrandPré,4.28


**Вывод** Наибольший средний показатель (4.28) у автора `J.K. Rowling/Mary GrandPré	`

### Запрос 5

**Формулировка задания** Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

**Комментарий к формулировке запроса** Запрос сформирован в следующем порядке:
- сформирована временная таблица содержащая сведения о количестве оценок выставленных каждым читателем, оставлены только те читатели, которые выставили более 50 оценок
- сформирована временная таблица содержащая сведения о количестве отзывов оставленных каждым читателем,
- временные таблицы соединены между собой
- выведены сведения о среднем количестве обзоров таких читателей

In [31]:
query = '''
        WITH 
        c_score AS (SELECT username as user, 
                            COUNT(rating_id) as count_score
                     FROM ratings 
                     GROUP BY username
                     HAVING COUNT(rating) > 50),
              
        c_review AS (SELECT username as user, 
                            COUNT(review_id) as count_review
                     FROM reviews AS rw
                     GROUP BY username)
              
        SELECT 
               AVG(count_review) as avg_count_reviews  
        FROM c_score
        LEFT JOIN c_review ON c_score.user=c_review.user  
        '''

data_table = view(query)
info = show_query(data_table)

,avg_count_reviews
0,24.33


**Вывод** В среднем, читатели, которые оставили более 50 оценок книгам написали немного больше 24 обзоров

## Общие выводы по работе с базой данных

В ходе проведенного исследования:
- удалось корректно подключиться к базе данных
- было проверено, что таблицы базы данных соответствуют описанию
- связи между таблицами корректные
- база данных пригодня для дальнейшей работы разрабатываемого сервиса
- в базе данных содержатся сведения о 1000 книг, их авторах, количестве страниц, издателе, дате публикации, оценках и отзывах читателей.